<a href="https://www.kaggle.com/code/karishmabattina/caloriepred-rforest?scriptVersionId=240171707" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [3]:
train_copy, test_copy = train_data, test_data

In [4]:
train_data.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [5]:
test_data.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [6]:
train_data.shape

(750000, 9)

In [7]:
test_data.shape

(250000, 8)

In [8]:
train_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [9]:
test_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64

In [10]:
train_data['BMI']=train_data['Weight']/((train_data['Height']/100) ** 2)
test_data['BMI']=test_data['Weight']/((test_data['Height']/100) ** 2)

In [11]:
gender_map = {"male" : 0,"female" : 1}
train_data['Sex'] = train_data['Sex'].replace(gender_map)

train_data=train_data.drop("id",axis=1)

print (train_data.head())

   Sex  Age  Height  Weight  Duration  Heart_Rate  Body_Temp  Calories  \
0    0   36   189.0    82.0      26.0       101.0       41.0     150.0   
1    1   64   163.0    60.0       8.0        85.0       39.7      34.0   
2    1   51   161.0    64.0       7.0        84.0       39.8      29.0   
3    0   20   192.0    90.0      25.0       105.0       40.7     140.0   
4    1   38   166.0    61.0      25.0       102.0       40.6     146.0   

         BMI  
0  22.955684  
1  22.582709  
2  24.690405  
3  24.414062  
4  22.136740  


/tmp/ipykernel_13/1709812675.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace(gender_map)


In [12]:
X = train_data.drop(['Calories'], axis=1)
y = train_data['Calories']

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
train_data, test_data

(        Sex  Age  Height  Weight  Duration  Heart_Rate  Body_Temp  Calories  \
 0         0   36   189.0    82.0      26.0       101.0       41.0     150.0   
 1         1   64   163.0    60.0       8.0        85.0       39.7      34.0   
 2         1   51   161.0    64.0       7.0        84.0       39.8      29.0   
 3         0   20   192.0    90.0      25.0       105.0       40.7     140.0   
 4         1   38   166.0    61.0      25.0       102.0       40.6     146.0   
 ...     ...  ...     ...     ...       ...         ...        ...       ...   
 749995    0   28   193.0    97.0      30.0       114.0       40.9     230.0   
 749996    1   64   165.0    63.0      18.0        92.0       40.5      96.0   
 749997    0   60   162.0    67.0      29.0       113.0       40.9     221.0   
 749998    0   45   182.0    91.0      17.0       102.0       40.3     109.0   
 749999    1   39   171.0    65.0      19.0        97.0       40.6     103.0   
 
               BMI  
 0       22.95568

In [16]:
rf_params = {'n_jobs': -1,
             'n_estimators': 150,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'max_features': 'log2',
             'max_depth': 20}

In [17]:
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)
y_pred = rf_model.predict(X_test)
print(f"RandomForest → R²: {r2_score(y_test, y_pred):.4f}, RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")

RandomForest → R²: 0.9965, RMSE: 3.70


In [18]:
test_data['Sex'] = test_data['Sex'].replace(gender_map)
test_ids = test_data['id']
X_test_final = test_data.drop('id', axis=1)


test_predictions = rf_model.predict(X_test_final)

submission = pd.DataFrame({
    'id': test_ids,
    'Calories': test_predictions
})
submission.to_csv('submission.csv', index=False)

/tmp/ipykernel_13/3580603541.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace(gender_map)


In [19]:
submission

,id,Calories
0,750000,27.335528
1,750001,105.576155
2,750002,86.885573
3,750003,126.892817
4,750004,75.625518
...,...,...
249995,999995,26.127046
249996,999996,9.194005
249997,999997,72.451083
249998,999998,168.898974
